<a href="https://colab.research.google.com/github/Darkprogrammerpb/Python_trainings/blob/master/pyspark_trials_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
########## Run this code snippet when running for the first time and don't repeat it in future (else it will keep on downloading the same stuffs again and again and
########## result in redundant usage of memory)

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apachemirror.wuchna.com/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

Code to initialise the pyspark session and loading libraries

In [0]:
import os
import findspark

import numpy as np
os.environ["JAVA_HOME"]   = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"]  = "/content/spark-2.4.4-bin-hadoop2.7"
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pandas as pd
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType
from pyspark.sql.types import *
spark                      = SparkSession.builder.master("local[*]").getOrCreate()

Loading the data to pyspark dataframe. We are reading a csv file in form of a pyspark dataframe and printing the first 2 rows

In [2]:
path = '/content/sample_data.csv'
df = spark.read.csv(path,header=True,inferSchema=True)
df.show(2)

+--------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
| Loan_ID|Gender|Married|Dependents|Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+--------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|LP001002|  Male|     No|         0| Graduate|           No|           5849|              0.0|      null|             360|             1|        Urban|          Y|
|LP001003|  Male|    Yes|         1| Graduate|           No|           4583|           1508.0|       128|             360|             1|        Rural|          N|
+--------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
only showing top

**Filtering a dataframe:-** We will filter the dataframe on basis of gender. We can pick single as well as multiple column as filters as shown below.

In [3]:
######## Filter a data frame based on just one condition #####################

df1 = df.filter(df['Gender']=='Male')
df2 = df.filter(df['Gender']=='Female')
df1.show(2)
df2.show(2)

+--------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
| Loan_ID|Gender|Married|Dependents|Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+--------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|LP001002|  Male|     No|         0| Graduate|           No|           5849|              0.0|      null|             360|             1|        Urban|          Y|
|LP001003|  Male|    Yes|         1| Graduate|           No|           4583|           1508.0|       128|             360|             1|        Rural|          N|
+--------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
only showing top

In [4]:
######## Here we do filtering based on multiple columns
df3 = df.filter((df['Gender']=='Male') & (df['Dependents']>0))
df3.show(2)

+--------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
| Loan_ID|Gender|Married|Dependents|Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+--------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|LP001003|  Male|    Yes|         1| Graduate|           No|           4583|           1508.0|       128|             360|             1|        Rural|          N|
|LP001011|  Male|    Yes|         2| Graduate|          Yes|           5417|           4196.0|       267|             360|             1|        Urban|          Y|
+--------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
only showing top

**Describing a particular Column:-** We use describe functionality to describe a particular column of a dataframe. We can apply describe to multiple columns as well

In [5]:
######## Applying describe on a particular column
df2.describe(['Loan_Amount_Term']).show()
####### Applying describe on multiple columns
df2.describe(['Loan_Amount_Term','Property_Area']).show()

+-------+------------------+
|summary|  Loan_Amount_Term|
+-------+------------------+
|  count|               109|
|   mean|352.29357798165137|
| stddev| 56.72208119782669|
|    min|                36|
|    max|               480|
+-------+------------------+

+-------+------------------+-------------+
|summary|  Loan_Amount_Term|Property_Area|
+-------+------------------+-------------+
|  count|               109|          112|
|   mean|352.29357798165137|         null|
| stddev| 56.72208119782669|         null|
|    min|                36|        Rural|
|    max|               480|        Urban|
+-------+------------------+-------------+



**Subsetting a dataframe based on some columns** Here we will be selecting only 5 columns of the dataframe and save them in another dataframe

In [6]:
columns  = df.columns
df4      = df.select(columns[1],columns[4],columns[0])
df4.show(3)

+------+---------+--------+
|Gender|Education| Loan_ID|
+------+---------+--------+
|  Male| Graduate|LP001002|
|  Male| Graduate|LP001003|
|  Male| Graduate|LP001005|
+------+---------+--------+
only showing top 3 rows



**Distinct Values in a column:-** We can use the distinct method to find distinct values in a particular dataframe column

In [7]:
df.select('Education').distinct().count()

2

**Dropping null values from a dataframe**:- We will drop null values from a dataframe using Dropna

In [8]:
new_df = df.dropna(how='any')
new_df.show(2)

+--------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
| Loan_ID|Gender|Married|Dependents|Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+--------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|LP001003|  Male|    Yes|         1| Graduate|           No|           4583|           1508.0|       128|             360|             1|        Rural|          N|
|LP001005|  Male|    Yes|         0| Graduate|          Yes|           3000|              0.0|        66|             360|             1|        Urban|          Y|
+--------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
only showing top

**GroupBy** We will by using groupby first using user defined functions first. We are taking Credit History as primary key for grouping and then applying aggregate functions accross the two columns mentioned- Dependents (max applied) and LoanAmount(average applied)

In [9]:
new_df.groupBy('Credit_History').agg({'Dependents':'max','LoanAmount':'avg'}).show()

+--------------+------------------+---------------+
|Credit_History|   avg(LoanAmount)|max(Dependents)|
+--------------+------------------+---------------+
|             1|143.38048780487804|             3+|
|             0|152.67142857142858|             3+|
+--------------+------------------+---------------+



In [10]:
### Applying a single function accross all the columns where it can be applied 
new_df.groupBy('Loan_Id').max().show()

+--------+--------------------+----------------------+---------------+---------------------+-------------------+
| Loan_Id|max(ApplicantIncome)|max(CoapplicantIncome)|max(LoanAmount)|max(Loan_Amount_Term)|max(Credit_History)|
+--------+--------------------+----------------------+---------------+---------------------+-------------------+
|LP002448|                3948|                1733.0|            149|                  360|                  0|
|LP002068|                4917|                   0.0|            130|                  360|                  0|
|LP002537|                2083|                3150.0|            128|                  360|                  1|
|LP001658|                3858|                   0.0|             76|                  360|                  1|
|LP001940|                3153|                1560.0|            134|                  360|                  1|
|LP002140|                8750|                4167.0|            308|                  360|    

**User Defined Functions in GroupBy:-** We will be using user defined functions in Groupby and hence we will introduce the concept of pandas UDF (user defined function). Here we are calculating the sum of all installments possible from the loan amount given in different Property areas (Urban, Rural and Semi Urban). The installment on one loan is attained by dividing the loan amount by loan term *(we are keeping things simple here and hence will just focus on a simple function for demonstration purpose.)*

In [11]:
@pandas_udf(FloatType(), functionType=PandasUDFType.GROUPED_AGG)
def per_inst_amt(loan_amount,loan_term):
  installments = loan_amount/loan_term
  return installments.sum()
  
new_df.groupBy("Property_Area").agg(per_inst_amt("LoanAmount", "Loan_Amount_Term").alias("Installment_sums")).show()  

+-------------+----------------+
|Property_Area|Installment_sums|
+-------------+----------------+
|        Urban|       65.167656|
|    Semiurban|       89.924164|
|        Rural|        65.35651|
+-------------+----------------+



**User defined Functions in Pyspark:-** We will create user defined functions here for the first time. It will take one(or more) columns as inputs and output a column.

---
Let us suppose we want to consider the monthly installment subject to each loan. and we provide an interest rate of 4% for Urban areas, 3% for semi urban areas and 2% for rural areas. We want to save the monthly installment in a new column -EMI

In [0]:
def calculate_monthly_installment(loan_amount,loan_term,property_area):
  if property_area =='Rural':
    return float(np.ppmt(0.02/12,1,loan_term/12.0,-1*float(loan_amount)))
  elif property_area == 'Semiurban':
    return float(np.ppmt(0.03/12,1,loan_term/12.0,-1*float(loan_amount)))
  else:
    return float(np.ppmt(0.04/12,1,loan_term/12.0,-1*float(loan_amount)))

In [0]:
from pyspark.sql.functions import udf
emi_udf = udf(calculate_monthly_installment, FloatType())

In [0]:
emidf = new_df.withColumn("EMI", emi_udf(new_df['LoanAmount'],new_df['Loan_Amount_Term'],new_df['Property_Area']))

In [31]:
emidf.select('Loan_ID','EMI','Loan_Amount_Term','LoanAmount','Property_Area').show()

+--------+----------+----------------+----------+-------------+
| Loan_ID|       EMI|Loan_Amount_Term|LoanAmount|Property_Area|
+--------+----------+----------------+----------+-------------+
|LP001003| 4.1644425|             360|       128|        Rural|
|LP001005| 2.0954945|             360|        66|        Urban|
|LP001006| 3.8099902|             360|       120|        Urban|
|LP001008| 4.4767385|             360|       141|        Urban|
|LP001011|  8.477228|             360|       267|        Urban|
|LP001013| 3.0162423|             360|        95|        Urban|
|LP001014| 5.0782127|             360|       158|    Semiurban|
|LP001018| 5.3339863|             360|       168|        Urban|
|LP001020| 11.217065|             360|       349|    Semiurban|
|LP001024| 2.2224944|             360|        70|        Urban|
|LP001028| 6.3499837|             360|       200|        Urban|
|LP001029| 3.7089567|             360|       114|        Rural|
|LP001030| 1.6746556|             120|  

**Joins in Pyspark Dataframe:-** We will create two datasets and perform joins on them

In [0]:
from pyspark.sql import *
Dataset1 = Row("primekey", "value1", "value2")
Dataset2 = Row("primekey", "value3", "value4")
data1   = [Dataset1('A', 12, 12),
            Dataset1('A', 14,16),
            Dataset1('B', 16,10),
            Dataset1('c', 20,22),
            Dataset1('c', 41,44)]
data2   = [Dataset2('A', 12, 16),
            Dataset2('B', 78,88),
            Dataset2('B', 99,21),
            Dataset2('D', 23,61),
            Dataset2('E', 42,11)]
df11 = spark.createDataFrame(data1)
df22 = spark.createDataFrame(data2)

In [0]:
df5 = df11.join(df22, on=['primekey'],how='inner')

In [61]:
df5.show(5)

+--------+------+------+------+------+
|primekey|value1|value2|value3|value4|
+--------+------+------+------+------+
|       B|    16|    10|    78|    88|
|       B|    16|    10|    99|    21|
|       A|    12|    12|    12|    16|
|       A|    14|    16|    12|    16|
+--------+------+------+------+------+



**Note(very important while performing Joins) :-** Always perform joins in the above syntax. There may be ways to join a dataframe using some specific methods as mentioned online but they all may suffer from a very subtle issue which causes errors while performing computations in future. Let me demonstrate via example

In [63]:
df5_1 = df11.join(df22, df11['primekey']==df22['primekey'])
df5_1.show()
print('The number of times the column -primekey appears using our current joining appreach is :- ',df5_1.columns.count('primekey'))
print('The number of times the column -primekey appeared in our previous joining approach is  :- ',df5.columns.count('primekey'))

+--------+------+------+--------+------+------+
|primekey|value1|value2|primekey|value3|value4|
+--------+------+------+--------+------+------+
|       B|    16|    10|       B|    78|    88|
|       B|    16|    10|       B|    99|    21|
|       A|    12|    12|       A|    12|    16|
|       A|    14|    16|       A|    12|    16|
+--------+------+------+--------+------+------+

The number of times the column -primekey appears using our current joining appreach is :-  2
The number of times the column -primekey appeared in our previous joining approach is  :-  1


We can clearly see that the current joining caused duplication of the column Education on which the join was made and hence it must be avoided.

In [64]:
### Sample demonstration of right join
df6 = df11.join(df22,on=['primekey'],how='right').show()

+--------+------+------+------+------+
|primekey|value1|value2|value3|value4|
+--------+------+------+------+------+
|       E|  null|  null|    42|    11|
|       B|    16|    10|    78|    88|
|       B|    16|    10|    99|    21|
|       D|  null|  null|    23|    61|
|       A|    12|    12|    12|    16|
|       A|    14|    16|    12|    16|
+--------+------+------+------+------+



**Union:-** Appending one dataframe at the end of other using union function. We will use union to append df1 and df2 (created previously) together

In [0]:
uniondf = df1.union(df2)

In [68]:
uniondf.show(3)
print('length of df1:- ',df1.count())
print('length of df2:- ',df2.count())
print('length of appended df:- ',uniondf.count())

+--------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
| Loan_ID|Gender|Married|Dependents|Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+--------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|LP001002|  Male|     No|         0| Graduate|           No|           5849|              0.0|      null|             360|             1|        Urban|          Y|
|LP001003|  Male|    Yes|         1| Graduate|           No|           4583|           1508.0|       128|             360|             1|        Rural|          N|
|LP001005|  Male|    Yes|         0| Graduate|          Yes|           3000|              0.0|        66|             360|             1|        Urban|          Y|
+--------+------